# Langchain Crash Course

In [5]:
from secret_key import openapi_key
# import os
# os.environ['OPENAI_API_KEY'] = openapi_key

import os
os.environ['OPENAI_API_KEY'] = openapi_key

# LLMS

In [6]:
from langchain_openai import ChatOpenAI

# Create an instance of the ChatOpenAI model
llm = ChatOpenAI(temperature=0.9)

# Use .invoke() instead of .predict()
name = llm.invoke("I want to open a restaurant for Indian food. Suggest a fancy name for this.")
print(name.content)


"Saffron Spice Palace"


In [7]:
llm.invoke("I want to open a restaurant for Indian food. Suggest a fancy name for this.").content


'"Spice Palace"'

# LLM response

In [8]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Define the prompt template
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

# Format the prompt
p = prompt_template_name.format(cuisine="Italian")
print("Prompt:", p)

# Initialize the LLM
llm = ChatOpenAI(temperature=0.9)

# Invoke the LLM and print the response
response = llm.invoke(p)
print("LLM Response:", response.content)


Prompt: I want to open a restaurant for Italian food. Suggest a fancy name for this.
LLM Response: "Bella Cucina Ristorante"


# Chain

In [11]:
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.9)
chain = LLMChain(llm=llm, prompt=prompt_template_name)
print(chain.invoke("Mexican"))


{'cuisine': 'Mexican', 'text': '"El Sabor Mexicano"'}


In [12]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
response = chain.invoke({"cuisine": "Mexican"})
print(response['text'])




> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fancy name for this.

> Finished chain.
"El Sabroso Cocina"


In [13]:
llm = ChatOpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables=['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}"
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)


# Simple Sequential Chain

In [14]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[name_chain, food_items_chain], verbose=True)

content = chain.invoke("Indian")
print(content)




> Entering new SimpleSequentialChain chain...
"Spice Palace"
1. Chicken Tikka Masala
2. Lamb Vindaloo
3. Vegetable Biryani
4. Paneer Butter Masala
5. Tandoori Chicken
6. Saag Paneer
7. Aloo Gobi
8. Butter Chicken
9. Dal Makhani
10. Garlic Naan

> Finished chain.
{'input': 'Indian', 'output': '1. Chicken Tikka Masala\n2. Lamb Vindaloo\n3. Vegetable Biryani\n4. Paneer Butter Masala\n5. Tandoori Chicken\n6. Saag Paneer\n7. Aloo Gobi\n8. Butter Chicken\n9. Dal Makhani\n10. Garlic Naan'}


# Sequential Chain

In [21]:
llm = ChatOpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)


In [26]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables=['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain = LLMChain(
    llm=llm,
    prompt=prompt_template_items,
    output_key="menu_items"  # Optional, but required if used in a multi-step chain
)


In [30]:
chain.invoke("Indian")




> Entering new SimpleSequentialChain chain...
"Spice Palace"
1. Chicken Tikka Masala
2. Lamb Vindaloo
3. Vegetable Biryani
4. Paneer Tikka
5. Butter Chicken
6. Aloo Gobi
7. Tandoori Chicken
8. Chana Masala
9. Naan bread
10. Mango Lassi

> Finished chain.


{'input': 'Indian',
 'output': '1. Chicken Tikka Masala\n2. Lamb Vindaloo\n3. Vegetable Biryani\n4. Paneer Tikka\n5. Butter Chicken\n6. Aloo Gobi\n7. Tandoori Chicken\n8. Chana Masala\n9. Naan bread\n10. Mango Lassi'}

# Agents

In [31]:
# make sure yoy have installed this package: pip install google-search-results
from secret_key import serpapi_key
# os.environ['SERPAPI_API_KEY'] = serpapi_key

os.environ['SERPAPI_API_KEY'] = serpapi_key

# serpapi and llm-math tool

In [43]:
from langchain_openai import OpenAI
from langchain.agents import AgentType, initialize_agent, load_tools

llm = OpenAI(temperature=0)

tools = load_tools(["serpapi","llm-math"],llm=llm)

agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

agent.invoke("What was the GDP of US in 2022 plus 5?")



> Entering new AgentExecutor chain...
 I should use the Calculator tool to solve this math problem.
Action: Calculator
Action Input: 2022 + 5
Observation: Answer: 2027
 Now I need to search for the GDP of the US in 2027.
Action: Search
Action Input: "GDP of US in 2027"
Observation: ['In the long-term, the United States GDP is projected to trend around 28756.00 USD Billion in 2026 and 29302.00 USD Billion in 2027, according to our econometric ...', "Such growth is 1.9 percent in 2025 and 1.8 percent in 2026 and 2027 in CBO's projections.", 'Release Table for 2027, Release Tables: Summary of Economic Projections. FRED: Download, graph, and track economic data.', 'Real gross domestic product (GDP) decreased at an annual rate of 0.3 percent in the first quarter of 2025 (January, February, and March), ...', "As of 2025, it has the world's seventh highest nominal GDP per capita and ninth highest GDP per capita by PPP. The U.S. accounted for 26% of the global economy ...", 'GDP, current pri

{'input': 'What was the GDP of US in 2022 plus 5?',
 'output': 'The GDP of the US in 2022 plus 5 is 29302.00 USD Billion.'}

In [45]:
llm = OpenAI(temperature=0)

tools = load_tools(["wikipedia","llm-math"],llm=llm)

agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

agent.invoke("When was Elon musk born? What is his age right now in 2025?")



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman known for his leadership of Tesla, SpaceX, and X (formerly Twitter). Since 2025, he has been a senior advisor to United States president Donald Trump and the de facto head of the Department of Government Efficiency (DOGE). Musk has been considered the wealthiest person in the world since 2021; as of March 2025, Forbes estimates his net worth to be US$345 billion. He was named Time magazine's Person of the Year in 2021.
Born to a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada. He graduated from the University of Pennsylvania in the U.S. before moving to California to pursue business ventures. In 1995, Musk co-founded the software company Zip2. Following its sale in 1999, he co-founded X.com, an onlin

{'input': 'When was Elon musk born? What is his age right now in 2025?',
 'output': 'Elon Musk will be 54 years old in 2025.'}

## Memory

In [48]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.invoke("Mexican")
print(name['text'])



"El Sabor de México" (The Flavor of Mexico)


In [50]:
name = chain.invoke("Indian")
print(name['text'])



"Spice Palace"


In [51]:
chain.memory

In [52]:
type(chain.memory)

NoneType

### ConversationBufferMemory

In [59]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True)
chain = LLMChain(llm=llm,prompt=prompt_template_name,memory=memory)
name  = chain.invoke({"cuisine":"Mexican"})
print(name)

{'cuisine': 'Mexican', 'history': [HumanMessage(content='Mexican', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\n"El Sabor de México" (The Flavor of Mexico)', additional_kwargs={}, response_metadata={})], 'text': '\n\n"El Sabor de México" (The Flavor of Mexico)'}


In [60]:
name = chain.invoke({"cuisine": "Arabic"})
print(name)

{'cuisine': 'Arabic', 'history': [HumanMessage(content='Mexican', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\n"El Sabor de México" (The Flavor of Mexico)', additional_kwargs={}, response_metadata={}), HumanMessage(content='Arabic', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\n"Al-Jannah Bistro"', additional_kwargs={}, response_metadata={})], 'text': '\n\n"Al-Jannah Bistro"'}


In [61]:
print(chain.memory.buffer)


[HumanMessage(content='Mexican', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\n"El Sabor de México" (The Flavor of Mexico)', additional_kwargs={}, response_metadata={}), HumanMessage(content='Arabic', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\n"Al-Jannah Bistro"', additional_kwargs={}, response_metadata={})]


In [62]:
print(chain.memory.chat_memory.messages)


[HumanMessage(content='Mexican', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\n"El Sabor de México" (The Flavor of Mexico)', additional_kwargs={}, response_metadata={}), HumanMessage(content='Arabic', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\n"Al-Jannah Bistro"', additional_kwargs={}, response_metadata={})]


## Conversation Chain

In [64]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.chat_history import InMemoryChatMessageHistory

# Define your LLM
llm = ChatOpenAI(temperature=0.7)

# Define the prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}"),
])

# Create a RunnableWithMessageHistory
runnable = RunnableWithMessageHistory(
    runnable=prompt | llm,
    get_session_history=lambda session_id: InMemoryChatMessageHistory()
)

# Simulate a session
session_id = "example-session"
response = runnable.invoke({"input": "Hello, how are you?"}, config={"configurable": {"session_id": session_id}})

print(response.content)


Hello! I'm here and ready to assist you. How can I help you today?


In [65]:
print(convo.run("Who won the first cricket world cup?"))
print(convo.run("How much is 5+5?"))
print(convo.run("Who was the captain of the winning team?"))


 The first cricket world cup was held in 1975 and was won by the West Indies team. They beat Australia in the final, with Clive Lloyd scoring a century and leading his team to victory. It was a historic moment for cricket and for the West Indies, as it was their first world cup win.
 5+5 is 10.
 The captain of the winning team was Clive Lloyd. He was a legendary cricketer and a key player in the West Indies team during that time. He also led the team to their second world cup win in 1979.


In [66]:
print(convo.memory.buffer)


Human: Who won the first cricket world cup?
AI:  The first cricket world cup was held in 1975 and was won by the West Indies team. They beat Australia in the final, with Clive Lloyd scoring a century and leading his team to victory. It was a historic moment for cricket and for the West Indies, as it was their first world cup win.
Human: How much is 5+5?
AI:  5+5 is 10.
Human: Who was the captain of the winning team?
AI:  The captain of the winning team was Clive Lloyd. He was a legendary cricketer and a key player in the West Indies team during that time. He also led the team to their second world cup win in 1979.


In [70]:
#Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}")
])

# Initialize LLM
llm = ChatOpenAI(temperature=0.7)

# Create the runnable with history support
runnable = RunnableWithMessageHistory(
    runnable=prompt | llm,
    get_session_history=lambda session_id: InMemoryChatMessageHistory()
)

# Use a consistent session_id for memory
session_id = "cricket-session"

# Run the chain
response = runnable.invoke(
    {"input": "Who won the first cricket world cup?"},
    config={"configurable": {"session_id": session_id}}
)

print(response.content)

The first Cricket World Cup was won by the West Indies in 1975.


In [72]:
memory = ConversationBufferWindowMemory(k=3)  # remembers last 3 exchanges
memory

ConversationBufferWindowMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), k=3)